# Training dormain specific Word2Vec word embeddings with Gensim

In [1]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
from nltk.tokenize import RegexpTokenizer
import multiprocessing

from gensim.models import Word2Vec

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
pd.set_option('max_colwidth', 1000)
pd.options.display.max_rows = 500
# Read file
df = pd.read_pickle('df_orig.pkl')

In [3]:
df.shape

(66806, 2)

In [4]:
df.head()

,Doc_Id,consumer_complaint_narrative
190126,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
190135,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
190155,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
190207,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
190208,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [5]:
df=df.reset_index(drop=True)

In [6]:
df.head()

,Doc_Id,consumer_complaint_narrative
0,0,XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! \nThey continue to insist I owe them and collection agencies are after me. \nHow can I stop this harassment for a bill I already paid four years ago? \n
1,1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in XXXX 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on XXXX X..."
2,2,"In XX/XX/XXXX my wages that I earned at my job decreased by almost half, by XX/XX/XXXX I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. \nIn early XX/XX/XXXX I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called XXXX. They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from XXXX and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork b..."
3,3,"I have an open and current mortgage with Chase Bank # XXXX. Chase is reporting the loan payments to XXXX but XXXX is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter XXXX BK discharged dated XXXX/XXXX/2013. Chase keeps referring to BK Law for Chapter XXXX and we keep providing documentation for Chapter XXXX, and the account should be open and current with all the payments \n"
4,4,"XXXX was submitted XX/XX/XXXX. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On XX/XX/XXXX Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in XX/XX/XXXX. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they ar..."


In [7]:
df['clean'] = df['consumer_complaint_narrative'].str.replace('X', '')
df['clean'] = df['clean'].str.replace('\n', '')

df = df.dropna().drop_duplicates()
df.shape


(66806, 3)

In [8]:
df_clean= df[['clean']]
df_clean.head()

,clean
0,has claimed I owe them {$27.00} for years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! They continue to insist I owe them and collection agencies are after me. How can I stop this harassment for a bill I already paid four years ago?
1,"Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan. In good faith, without having received any proof as to why there was still a balance, I made a partial payment towards the remaining debt. I then sent the goodwill letter still offering to pay the remainder of the debt, but in exchange for the removal of the negative entry on my credit report. At one point, in 2015, per my credit monitoring agency, it showed a delinquent balance of {$0.00}, but when I checked my credit report again on 2015, th..."
2,"In // my wages that I earned at my job decreased by almost half, by // I knew I was in trouble with my home loan. I began contacting WFB whom my home loan is with, for assitance and options. In early // I began the Loan Modification process with Wells Fargo Bank. I was told that they would not assist me with anything financial on my home loan until I fell 90 days behind, though at the time I started to inquire for assistance from WFB I was only a few weeks behind. So, I began working with a program called . They approved me for a variety of assistence and reached out to Wells Fargo Bank to determine what they could assist with. Wells Fargo then turned down the assistance from and finally offered to do a Loan Modification for me. The outcome was totally unknow about what I would be offered in the end by WFB for assistance. Wells Fargo lost my paperwork twice during this process, so it took 2 months from the time I started to the time my paperwork began to be processed for some kind..."
3,"I have an open and current mortgage with Chase Bank # . Chase is reporting the loan payments to but is surpressing the information and reporting the loan as Discharged in BK. This mortgage was reaffirmed in a Chapter BK discharged dated //2013. Chase keeps referring to BK Law for Chapter and we keep providing documentation for Chapter , and the account should be open and current with all the payments"
4,"was submitted //. At the time I submitted this complaint, I had dealt with Rushmore Mortgage directly endeavoring to get them to stop the continuous daily calls I was receiving trying to collect on a mortgage for which I was not responsible due to bankruptcy. They denied having knowledge of the bankruptcy, even though I had spoken with them about it repeatedly and had written them repeatedly referencing the bankruptcy requesting them to cease the pursuit, they continued to do so. When they were unable to trick me into paying, force me into paying in retaliation they placed reported to my credit bureaus a past due mortgage amount that had been discharged in Federal Court. On // Rushmore responded the referenced complaint indicating that they would remove the reporting from my bureau, yet it is still there now in //. I would like them to remove it immediately and send me a letter indicating that it should not have been there in the first place and they are going to remove it from al..."


In [9]:
sentences = df_clean.clean.astype('str').tolist()
sentences[0]

' has claimed I owe them {$27.00} for  years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! They continue to insist I owe them and collection agencies are after me. How can I stop this harassment for a bill I already paid four years ago? '

In [10]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in sentences]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

In [11]:
sentences_tokenized[0]

['has',
 'claimed',
 'i',
 'owe',
 'them',
 '27',
 '00',
 'for',
 'years',
 'despite',
 'the',
 'proof',
 'of',
 'payment',
 'i',
 'sent',
 'them',
 'canceled',
 'check',
 'and',
 'their',
 'ownpaid',
 'invoice',
 'for',
 '27',
 '00',
 'they',
 'continue',
 'to',
 'insist',
 'i',
 'owe',
 'them',
 'and',
 'collection',
 'agencies',
 'are',
 'after',
 'me',
 'how',
 'can',
 'i',
 'stop',
 'this',
 'harassment',
 'for',
 'a',
 'bill',
 'i',
 'already',
 'paid',
 'four',
 'years',
 'ago']

Most Frequent Words:

In [12]:
word_freq = defaultdict(int)
for sents in sentences_tokenized:
    for i in sents:
        word_freq[i] += 1
len(word_freq)

52406

In [13]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

4

In [14]:
w2v_model = Word2Vec(min_count=10,
                     window=10,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [15]:
t = time()

w2v_model.build_vocab(sentences_tokenized, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:22:17: collecting all words and their counts
INFO - 21:22:17: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:22:17: PROGRESS: at sentence #10000, processed 1814017 words, keeping 20805 word types
INFO - 21:22:18: PROGRESS: at sentence #20000, processed 3578878 words, keeping 29018 word types
INFO - 21:22:18: PROGRESS: at sentence #30000, processed 5368922 words, keeping 35660 word types
INFO - 21:22:19: PROGRESS: at sentence #40000, processed 7188524 words, keeping 41580 word types
INFO - 21:22:19: PROGRESS: at sentence #50000, processed 9005746 words, keeping 46476 word types
INFO - 21:22:19: PROGRESS: at sentence #60000, processed 10792301 words, keeping 50097 word types
INFO - 21:22:19: collected 52406 word types from a corpus of 12055423 raw words and 66806 sentences
INFO - 21:22:19: Loading a fresh vocabulary
INFO - 21:22:20: effective_min_count=10 retains 11539 unique words (22% of original 52406, drops 40867)
INFO - 21:22:20: effective_min_

Time to build vocab: 0.05 mins


In [16]:
t = time()

w2v_model.train(sentences_tokenized, total_examples=w2v_model.corpus_count, epochs=5, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:22:20: training model with 3 workers on 11539 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=10
INFO - 21:22:21: EPOCH 1 - PROGRESS: at 4.54% examples, 181636 words/s, in_qsize 6, out_qsize 0
INFO - 21:22:22: EPOCH 1 - PROGRESS: at 9.99% examples, 195257 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:23: EPOCH 1 - PROGRESS: at 15.64% examples, 204795 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:24: EPOCH 1 - PROGRESS: at 21.60% examples, 210782 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:25: EPOCH 1 - PROGRESS: at 27.22% examples, 213008 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:26: EPOCH 1 - PROGRESS: at 33.19% examples, 215395 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:27: EPOCH 1 - PROGRESS: at 38.79% examples, 216761 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:28: EPOCH 1 - PROGRESS: at 44.27% examples, 216999 words/s, in_qsize 5, out_qsize 0
INFO - 21:22:29: EPOCH 1 - PROGRESS: at 49.33% examples, 215254 words/s, in_qsize 5, out_q

Time to train the model: 1.52 mins


In [17]:
# generic word2vec model
w2v_model.wv.save_word2vec_format("cfpb_generic_word2vec.model")

INFO - 21:23:51: storing 11539x300 projection weights into cfpb_generic_word2vec.model


In [18]:
w2v_model.wv.most_similar(positive=["delinquent"],topn=20)

INFO - 21:23:54: precomputing L2-norms of word weight vectors
C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('deliquent', 0.7170357704162598),
 ('delinquency', 0.59906005859375),
 ('late', 0.5936636924743652),
 ('overdue', 0.5436649322509766),
 ('reported', 0.5293423533439636),
 ('delinquencies', 0.5287643671035767),
 ('missed', 0.49839580059051514),
 ('past', 0.4968588650226593),
 ('lates', 0.49249428510665894),
 ('due', 0.48416903614997864),
 ('120', 0.48219630122184753),
 ('derogatory', 0.47709208726882935),
 ('erroneously', 0.4717191457748413),
 ('partial', 0.4580209255218506),
 ('incorrectly', 0.45547181367874146),
 ('reflecting', 0.4457855820655823),
 ('unpaid', 0.4451330304145813),
 ('default', 0.44133076071739197),
 ('statuses', 0.4385424554347992),
 ('status', 0.43819600343704224)]

In [19]:
w2v_model.wv.most_similar(positive=["mod"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('modification', 0.8667501211166382),
 ('hamp', 0.7372881174087524),
 ('reapply', 0.7240679264068604),
 ('modifications', 0.7027243375778198),
 ('modified', 0.6796156167984009),
 ('trial', 0.6725947856903076),
 ('remodification', 0.6690160036087036),
 ('pkg', 0.6637154221534729),
 ('modify', 0.6528689861297607),
 ('resubmit', 0.6330093145370483),
 ('finalized', 0.623599648475647),
 ('packet', 0.6232447624206543),
 ('paperwork', 0.6183032989501953),
 ('permanent', 0.6139217019081116),
 ('modifying', 0.6106398105621338),
 ('rma', 0.6007360219955444),
 ('packets', 0.5978703498840332),
 ('docs', 0.5924543142318726),
 ('proposal', 0.5863274931907654),
 ('modifcation', 0.5836915373802185)]

In [20]:
w2v_model.wv.most_similar(positive=["ohio"],topn=20)

C:\Users\alber\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('indiana', 0.7717363834381104),
 ('illinois', 0.7608502507209778),
 ('alabama', 0.7381572723388672),
 ('georgia', 0.7323154807090759),
 ('oklahoma', 0.7272956967353821),
 ('michigan', 0.7262011766433716),
 ('louisiana', 0.7190572023391724),
 ('maryland', 0.7189023494720459),
 ('kansas', 0.7176732420921326),
 ('minnesota', 0.7164055109024048),
 ('nevada', 0.7156771421432495),
 ('virginia', 0.7096671462059021),
 ('nj', 0.7053618431091309),
 ('pennsylvania', 0.6934981346130371),
 ('colorado', 0.6924794912338257),
 ('carolina', 0.6921257972717285),
 ('mississippi', 0.6917270421981812),
 ('south', 0.6907212734222412),
 ('ny', 0.6822537779808044),
 ('florida', 0.6807726621627808)]

In [21]:
len(w2v_model.wv.vocab)

11539